# DSCI 512 Lab 3




### Instructions
rubric={mechanics:5}

Follow the [general lab instructions](https://ubc-mds.github.io/resources_pages/general_lab_instructions/).

In [ ]:
import numpy as np
import pandas as pd
import altair as alt
import networkx as nx
import matplotlib.pyplot as plt
from collections import defaultdict

## Exercise 1: The Romeo and Juliet graph

Consider this graph of the social network connections in _Romeo and Juliet_ (you can ignore the gray background and the labels for the families):

![](romeo-and-juliet-undirected.png)

Image attribution: the Romeo and Juliet graph, and inspiration, taken from [UW CSE 140](https://courses.cs.washington.edu/courses/cse140/13wi/homework/hw4/homework4.html) with permission.

The code below creates a graph named `rj` corresponding to the Romeo and Juliet graph above.

In [ ]:
rj = nx.Graph()
rj.add_nodes_from(['Nurse',
                   # House of Capulet
                   'Juliet', 'Tybalt', 'Capulet',

                   'Friar Laurence',

                   # House Montague
                   'Romeo', 'Benvolio', 'Montague',

                   # Ruling house of Verona
                   'Escalus', 'Mercutio', 'Paris'
                   ])

rj.add_edges_from([('Juliet', 'Nurse'),
                   ('Juliet', 'Tybalt'),
                   ('Juliet', 'Capulet'),
                   ('Juliet', 'Friar Laurence'),
                   ('Juliet', 'Romeo'),

                   ('Capulet', 'Tybalt'),
                   ('Capulet', 'Escalus'),
                   ('Capulet', 'Paris'),

                   ('Romeo', 'Friar Laurence'),
                   ('Romeo', 'Benvolio'),
                   ('Romeo', 'Montague'),
                   ('Romeo', 'Mercutio'),

                   ('Montague', 'Benvolio'),
                   ('Montague', 'Escalus'),

                   ('Escalus', 'Mercutio'),
                   ('Escalus', 'Paris'),
                   ('Paris', 'Mercutio')
                   ])
nx.draw(rj, with_labels=True)

#### 1(a) (optional) graphs warmup
rubric={accuracy:1}

Write a function `highest_degree` that takes in a graph and finds the vertex/vertices with the highest degree - in other words, the person with the largest number of friends. Your function should return a tuple with two elements:

1. The maximum degree (int)
2. All the nodes with that degree (set)

As always, a proper docstring is required.

Note: you can find the degree of a vertex using the following syntax:

In [ ]:
nx.degree(rj, "Paris")

and you can iterate through all nodes in a graph like this:

In [ ]:
for node in rj.nodes():
    print(node)

In [ ]:
max_degree, nodes = highest_degree(rj)
assert nodes == {"Romeo", "Juliet"}
assert max_degree == 5

#### 1(b) largest distance
rubric={accuracy:4,quality:2}

One interesting measure in a social network graph is the "distance" or number of "degrees of separation" between two people. This notion is used in academic research via the [Erdős number](https://en.wikipedia.org/wiki/Erd%C5%91s_number) and in the film industry via the [Bacon number](https://en.wikipedia.org/wiki/Six_Degrees_of_Kevin_Bacon#Bacon_numbers). For example, in the above graph, the distance between Juliet and Romeo is 1, and the distance between Juliet and Paris is 2 (via Capulet).


In [ ]:
nx.shortest_path_length(rj, "Juliet", "Romeo")

In [ ]:
nx.shortest_path_length(rj, "Juliet", "Paris")

Write a function `largest_distance` to find the pair(s) of vertices with the largest distance (degree of separation). Your function should return a tuple with two elements:

1. The maximum distance (int)
2. All the pairs of nodes with that distance (set of tuples)

Note: do not include pairs twice. For example, if `('Romeo', 'Juliet')` is in the set, don't also include `('Juliet', 'Romeo')`.

Your function should work by iterating through all pairs of vertices. For each pair, it should compute the distance (degree of separation) between nodes using the function `nx.shortest_path_length` used above. Later, we'll try something else that will be better.

In [ ]:
def largest_distance(G):
    """
    Finds the pairs of vertices with the highest degree of separation in the graph G.
    Returns a tuple (max_distance, {node pairs with that distance})

    Parameters
    ----------
    G : networkx.classes.graph.Graph
        a network graph

    Returns
    -------
    max_distance: int 
       the maximum connections separating nodes 
    node pairs with max degree distance: set 
        nodes with largest separation  

    Examples
    --------
    >>> test = nx.Graph()
    >>> test.add_nodes_from(['A', 'B', 'C', 'D'])
    >>> test.add_edges_from([('A', 'B'), ('B', 'C'),  ('C', 'D'), ('A', 'D')])
    >>> largest_distance(test)
    (2, {('A', 'C'), ('B', 'D')})
    """
    
    # YOUR CODE HERE
    

In [ ]:
max_distance_rj, pairs_rj = largest_distance(rj)
assert max_distance_rj == 3
assert len(pairs_rj) == 12

In [ ]:
test = nx.Graph()
test.add_nodes_from(['A', 'B', 'C', 'D'])
test.add_edges_from([('A', 'B'), ('B', 'C'),  ('C', 'D'), ('A', 'D')])
max_distance_test, pairs_test = largest_distance(test)
assert max_distance_test == 2
assert pairs_test == {('A', 'C'), ('B', 'D')}

#### 1(c)
rubric={reasoning:2}

Assuming that `nx.shortest_path_length` takes $O(V)$ time in the worst case, what is the worst case time complexity if your `largest_distance` function from the previous part? Justify your answer.

## Exercise 2: computing degrees of separation with BFS
rubric={accuracy:7}

Let's see if we can come up with a better way to find the largest distance between two nodes in a graph; that is, an approach with a better time complexity than what we saw in the previous part. 

Here is the reason why the previous approach is unnecessarily slow: in calling `nx.shortest_path_length` between pairs of nodes we do a bunch of redundant/repeated computation. For example, computing the distance between Nurse and Mercutio is fairly similar to finding the distance between Juliet and Mercutio, but we're doing it all from scratch for every pair of nodes.

Below we provide a function `distance_BFS` that takes in a networkx `Graph` and two nodes, and uses breadth-first search (BFS) to compute the distance between the two nodes; in other words, it does the same thing as `nx.shortest_path_length`. The function returns -1 if the two nodes are not connected (whereas `nx.shortest_path_length` throws an error). The code here is similar to the BFS code from lecture, except that instead of only storing the nodes in the queue, we store tuples of the (node, distance) so that we can keep track of the distance.

In [ ]:
def distance_BFS(G, node1, node2):
    """ 
    Given a NetworkX Graph G, and start node node1 
    and goal node node2, distance_BFS returns the
    degree of separation between node1 and node2. 

    Parameters
    ----------
    G : networkx.classes.graph.Graph
        the graph
    node1 : str
        first node
    node2 : str 
        second node

    Returns
    -------
    int 
        the distance between 2 nodes, if
        they are not connected, returns -1

    Examples
    --------
    >>> test = nx.Graph()
    >>> test.add_nodes_from(['A', 'B', 'C', 'D'])
    >>> test.add_edges_from([('A', 'B'), ('B', 'C'),  ('C', 'D'), ('A', 'D')])
    >>> distance_BFS(test, 'A', 'C')
    2
    """

    queue = [(node1, 0)]
    visited = {node1}

    while queue:
        vertex, distance = queue.pop(0)
        if vertex == node2:
            return distance

        for neighbour in G.neighbors(vertex):
            if neighbour not in visited:
                queue.append((neighbour, distance + 1))
                visited.add(neighbour)
    return -1

Some tests:

In [ ]:
assert(distance_BFS(rj, "Juliet", "Romeo")) == 1
assert(distance_BFS(rj, "Juliet", "Paris")) == 2
assert(distance_BFS(rj, "Nurse", "Paris")) == 3
assert(distance_BFS(rj, "Nurse", "Mercutio")) == 3

In [ ]:
rj2 = rj.copy()
rj2.add_node("Santa Claus")

In [ ]:
assert(distance_BFS(rj2, "Romeo", "Santa Claus")) == -1

Your task is to adapt/modify the above code to create a function that takes in a node and returns the furthest (largest distance) node. The changes to the above code are not that major - you shouldn't be rewriting it from scratch. I suggest you start by pasting in the body of `distance_BFS` and modifying it from there.

Your code should run in $O(V)$ time.

In [ ]:
def furthest_from_node(G, node):
    """ 
    Find the furthest node from the input node in a Graph G.

    Parameters
    ----------
    G : networkx.classes.graph.Graph
        the graph
    node : str
        the node

    Returns
    -------
    tuple (int, node)
        the first element is the largest distance
        the second element is a set of the nodes that achieve this distance

    Examples
    --------
    >>> test = nx.Graph()
    >>> test.add_nodes_from(['A', 'B', 'C', 'D'])
    >>> test.add_edges_from([('A', 'B'), ('B', 'C'),  ('C', 'D'), ('A', 'D')])
    >>> furthest(test, 'A')
    2
    """

    # YOUR CODE HERE

In [ ]:
assert furthest_from_node(test, 'A')[0] == 2
assert furthest_from_node(test, 'A')[1] == {'C'}

In [ ]:
assert furthest_from_node(rj, 'Nurse')[0] == 3
assert furthest_from_node(rj, 'Nurse')[1] == {'Paris', 'Escalus', 'Mercutio', 'Benvolio', 'Montague'}

assert furthest_from_node(rj, 'Juliet')[0] == 2
assert furthest_from_node(rj, 'Nurse')[1] == {'Paris', 'Escalus', 'Mercutio', 'Benvolio', 'Montague'}

## Exercise 3: code timing

Assuming you have implemented `furthest_from_node` above, the function `largest_distance_faster` below will call your `furthest_from_node` function and find the pair of nodes with the largest distance. The code isn't very pretty, but it should work the same way as your `largest_distance` from 1(b) - but faster!

In [ ]:
def largest_distance_faster(G):
    """
    Finds the pairs of vertices with the highest degree of separation in the graph G.
    Returns a tuple (max_distance, {node pairs with that distance})

    Parameters
    ----------
    G : networkx.classes.graph.Graph
        a network graph

    Returns
    -------
    max_distance: int 
       the maximum connections separating nodes 
    node pairs with max degree distance: set 
        nodes with largest separation  

    Examples
    --------
    >>> test = nx.Graph()
    >>> test.add_nodes_from(['A', 'B', 'C', 'D'])
    >>> test.add_edges_from([('A', 'B'), ('B', 'C'),  ('C', 'D'), ('A', 'D')])
    >>> largest_distance(test)
    (2, {('A', 'C'), ('B', 'D')})
    """

    overall_max_dist = 0
    distances = dict()
    for v in G.nodes():
        max_dist, max_dist_nodes = furthest_from_node(G, v)

        distances[v] = (max_dist, max_dist_nodes)
        overall_max_dist = max(overall_max_dist, max_dist)
    
    node_pairs = set()
    for v, output in distances.items():
        output_dist, output_nodes = output
        if output_dist == overall_max_dist:
            for w in output_nodes:
                if (w, v) not in node_pairs:
                    node_pairs.add((v, w))
    
    return overall_max_dist, node_pairs

In [ ]:
assert largest_distance_faster(test)[0] == 2
assert largest_distance_faster(test)[1] == {('A', 'C'), ('B', 'D')}

In [ ]:
max_distance, pairs = largest_distance(rj)
assert max_distance == 3
assert len(pairs) == 12

#### 3(a)
rubric={reasoning:2}

What is the time complexity of `largest_distance_faster`?

#### 3(b)
rubric={reasoning:6}

Here's a networkx function that generates graphs:

In [ ]:
ladder = nx.generators.classic.ladder_graph(3)
nx.draw(ladder)

You can change the size by changing its argument:

In [ ]:
ladder = nx.generators.classic.ladder_graph(5)
nx.draw(ladder)

Using this function to generate graphs, perform a timing experiment to time `largest_distance` (which you wrote) and `largest_distance_faster` (which I wrote, but which calls `furthest_from_node` that you wrote). Are the results consistent with the big-O running times we expected? Is `largest_distance_faster` indeed faster than `largest_distance`? 

#### (optional) 3(c)
rubric={reasoning:1}

`networkx` also has many functions for creating random graphs. Here is one:

In [ ]:
rnd = nx.generators.random_graphs.connected_watts_strogatz_graph(5, 2, 0.2)
nx.draw(rnd)

The first argument is the number of vertices. E.g. we can expand it from 5 to 10:

In [ ]:
rnd = nx.generators.random_graphs.connected_watts_strogatz_graph(10, 2, 0.2)
nx.draw(rnd)

If you generate the graphs with `nx.generators.random_graphs.connected_watts_strogatz_graph` instead, does that change anything from the previous part? Can you explain why?

## Exercise 4: Using Facebook data

We will use some Facebook data, which is the file `facebook-links.txt` in the [datasets repository](https://github.ubc.ca/MDS-2020-21/datasets). Note: if you go to that file in the repo, and right-click "Download", and select Save File As (or similar), then you can get this file without cloning/downloading the whole datasets repo.

The `facebook-links.txt` file is courtesy of the Max Planck Institute for Software Systems. Here is a slightly clarified version of the documentation for this file:

> File `facebook-links.txt` contains a list of all of the user-to-user links from the Facebook New Orleans networks. These links are undirected on Facebook.
>
> Format: Each line contains two numeric user identifiers, meaning the second user appeared in the first user's friend list, and the first user appeared in the second user's friend list. Finally, the third column is a UNIX timestamp with the time of link establishment (if it could be determined, otherwise it is '\N'). Note: a Unix timestamp is the number of seconds since January 1, 1970. You may ignore it in this assignment. 


The code below creates a graph named facebook from the Facebook data in file `facebook-links.txt`. 

In [ ]:
facebook_edges = pd.read_csv("facebook-links.txt", sep='\t')
facebook_edges.columns = ["from", "to", "timestamp"]
facebook_edges.head()

In [ ]:
facebook_graph = nx.convert_matrix.from_pandas_edgelist(
    facebook_edges, "from", "to")

In [ ]:
assert len(facebook_graph.nodes()) == 63731
assert len(facebook_graph.edges()) == 817090

#### (optional) 4(a)
rubric={viz:1}

Using your plotting skills from DSCI 531, create a histogram of the node degrees in this graph. For example, how many nodes have degree 1, how many nodes have degree 2, etc. Make sure you use enough bins so you can see the general shape of things.

Note: you an get the degree of a node with `graph.degree(node)`.

#### (optional) 4(b)
rubric={accuracy:1}

To summarize the histogram further, find the mean, median, and mode of the vertex degrees.

#### 4(c)
rubric={reasoning:1}

Although all nodes in this dataset have at least one friend, the graph is not _connected_; that is, for two nodes A and B, there is not necessarily a path from A to B. A subset of nodes that are connected is called a _connected component_. We can get the connected components using `nx.connected_components(graph)` in networkx.

To help you understand what `connected_components` returns, let's take this example graph from lecture:

In [ ]:
disconnected = nx.Graph()

disconnected.add_edge("A", "B")
disconnected.add_edge("A", "C")
disconnected.add_edge("D", "C")
disconnected.add_edge("B", "C")

disconnected.add_edge("E", "F")

nx.draw(disconnected, with_labels=True)

Here is the output of

In [ ]:
list(nx.connected_components(disconnected))

As we can see, it returns a generator (casted to a list above) of sets, each containing a connected component in the graph.

Let's look at the number of vertices in these connected components from the Facebook graph:

In [ ]:
components = nx.connected_components(facebook_graph)

In [ ]:
num_verts = [len(cc) for cc in components]

In [ ]:
np.array(num_verts)

In 1-2 sentences, briefly discuss the result above in the context of the social network. What is going on here?

#### 4(d)
rubric={reasoning:2}

Below we grab the largest connected component of the Facebook graph.

In [ ]:
connected_components = nx.connected_components(facebook_graph)
fb_largest_cc_nodes = next(connected_components)
fb_largest_cc = facebook_graph.subgraph(fb_largest_cc_nodes).copy()

Here is the number of nodes:

In [ ]:
len(fb_largest_cc)

A lot of this lab has been about finding the pair of nodes with the largest distance between them. Let's say you wanted to find the largest distance between any pair of nodes in the largest connected component of the Facebook graph. With such a large graph, this might take a long time. Using your timing analysis from 3(b) above, estimate how long this would take using `largest_distance_faster`.

#### (optional) 4(e)
rubric={reasoning:1}



Do your best to estimate the largest distance between nodes in the Facebook connected component. You can use any means at your disposal, including waiting a long time for the code to run. There are a lot of ways to approach this question!

#### 4(f)
rubric={reasoning:1}

How many bytes do you think the dense (not sparse) form of the Facebook adjacency matrix would take, if each element takes 8 bytes? (If you try it, you may well run out of memory).

#### 4(g)
rubric={reasoning:1}

How many bytes do you think the sparse form of the Facebook adjacency matrix would take, if each element takes 8 bytes?

## (optional) Exercise 5: assessing virality
rubric={reasoning:1}

Everyone wants their video or app to "go viral". This can occur by something spreading through a social network. Here, we will model virality as follows: 

1. pick some virality coefficient $\xi\in (0,1)$
2. select one person (node) at random to be initially "infected"
3. each currently infected person loses interest with probability $\alpha$ and becomes _permanently_ un-infected. By default we'll use $\alpha=0.01$.
4. for each infected person, each neighbour in the graph becomes infected with probability $\xi$. Note: if multiple neighbours of an un-infected node are infected, repeat this step multiple times. For example, if Mercutio and Paris like _Gangnam Style_, then Escalus has two chances of being infected at the current time step. Mathematically, the probability of infection is $1-(1-\xi)^2 = 2\xi-\xi^2$, but you don't need to calculate this in your code because you can just repeatedly try to infect the person.
5. repeat steps 3-4 some number of times, by default $1000$. 

Write a function implementing this model. Your function should return a list/array of the proportion of people infected at each iteration. Using the Facebook data set (or a subgraph if the code is too slow), explore some or all of the following questions:

1. Investigate how the number of infected people proceeds as a function of time: what is the general shape you observe? Is it consistent across runs of the simulation?
2. Report the _maximum_ proportion of your population that was infected at any given time. Try this for a couple values of $\xi$ and investigate how the maximum proportion of infected people depends on $\xi$. Note: For a given value of $\xi$ you will need to run several simulations and average the results to reduce noise. 
3. Do your results depend significantly on the connectivity of the graph? For example, if you randomly remove half of the edges in the Facebook graph (be careful not to delete any vertices in the process!), what is the effect on the virality? 

Note: this model for virality is similar to rudimentary models of how diseases spread across populations. See [here](https://en.wikipedia.org/wiki/Mathematical_modelling_of_infectious_disease#The_SIR_model).

## (optional) Exercise 6: setting priorities and time-management skills
rubric={reasoning:100}

_Rationale: admission into the MDS program is very competitive. This is great for me, because I enjoy working with such motivated and talented individuals. However, I conjecture that this same admissions process may also select for people with unrealistically high expectations of themselves. For example, I have noticed some students feel they must do all the optional questions even if they aren't particularly interested in the topic or don't really have the time to do them. If that sounds like you, this optional exercise was created for you! My hope is that none of you attempts, let alone completes, this silly exercise. In skipping it, you will need to forego the 100 bonus points.$^*$ I hope that by doing so, you will feel that it's perfectly fine to triage and skip certain lab questions: nothing bad happens! I believe this skill --- or perhaps you could call it a mindset --- will be extremely important for your success and wellbeing in the long-term. Doing everything perfectly is simply not possible forever, and when that time comes, it is important that you can set priorities. I have seen many people be ineffective at their jobs because they cannot skip the unimportant things. And now, the task..._

Do the following before the lab deadline:

- 400 push-ups
- 300 sit-ups
- recite the alphabet backwards 200 times
- memorize the names of 100 countries
- ask someone the following question: "What is $\int_1^\text{cabin} \frac{1}{x}\text{d}x$?"

You must include, with your submission, incontrovertible evidence that you did each of these things. Your word of honour will not be sufficient.

$^*$Also, please note that all lab grades in MDS are capped at 100%. That is, you can increase your grade up to 100% with optional questions, but not beyond 100%.

#### (optional) 6(b)
rubric={reasoning:1}

If you completed the first part of this exercise, write a brief reflection in 2-3 sentences. What did you learn about setting priorities and time-management skills?